# SSH

This page shows aspects of working with the ssh connection. We will consider an example where an ssh connection is established from one Docker container to another.

## Set up

Let's create the necessary docker entities:

- `ssh_host` container that starts the ssh server - we would be able to connect to this server using ssh.
- `ssh_client` container to which we will give access to the `ssh_host`.
- `network` to allow communication between containers.

In [2]:
docker network create ssh_test_network
docker run -dit --rm --name ssh_host --network ssh_test_network alpine
docker run -dit --rm --name ssh_client --network ssh_test_network alpine

1567737275c313c43b56b8fafa7708199fc569c760b59bec77b549e404212769
c820ca94c2280eb4401c957f541d74507b6712ad22d2d3881e6fe25891f2f449
0f0391a4e1a5e54b1cf43d34e7bc0c92fc917683175aec39dd2d69ca33c63382


Install and start the ssh server.

In [3]:
docker exec -d ssh_host sh -c " 
# openssh instalation
apk add -q openssh
# generating default host keys - they are necessary for running ssh server
ssh-keygen -A
# running ssh server
/usr/sbin/sshd -D"

Installing ssh client.

In [4]:
docker exec ssh_client apk add -q openssh-client

**Note** Don't forget to stop the container and the network.

In [1]:
docker stop ssh_client ssh_host
docker network rm ssh_test_network

ssh_client
ssh_host
ssh_test_network


## Generating key

You need to generate an ssh key. It consists of two parts, a public key and a private key. You can generate it wherever you like. But in our example we will consider the most straightforward option - generating it on the client.

You can just run `ssh-keygen` and follow the CLI's instructions. It'll ask for filepath for the key and passphrase. However, due to the limitations of the jupyter notebook we are using for the presentation, we need to specify some options:

- `t rsa`: This option specifies the type of key algorithm to use. In this case, it specifies RSA as the algorithm. RSA is a widely used asymmetric encryption algorithm.
- `N ""` : This option sets the passphrase (or password) for the generated key. In your command, it is set to an empty string, effectively creating a key without a passphrase. It means that anyone with access to the private key file can use it without providing a passphrase.
- `f /root/.ssh/id_rsa`: This option specifies the filename and path where the generated key pair will be stored. In this case, it will be stored in the /root/.ssh/ directory with the filename id_rsa. The id_rsa file will contain the private key, while the id_rsa.pub file (automatically generated) will contain the public key.


In [5]:
docker exec ssh_client ssh-keygen -t rsa -N "" -f /root/.ssh/id_rsa

Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:qr6Sz5Nh8LuK4dz569OTX9m76UtLT8M8CoM9SJ3eq0c root@0f0391a4e1a5
The key's randomart image is:
+---[RSA 3072]----+
|                 |
|                 |
|                 |
|  .       . .    |
|   o    S. o     |
|    +  .. = +Eo  |
|.  o +o .o O.= * |
|oo+.=o +  . *.O o|
|.o.BXBo o. .+Xo. |
+----[SHA256]-----+


After all I'll have `id_rsa` and `id_rsa.pub` files in my the sepcified for ssh keys folder.

In [6]:
docker exec ssh_client ls /root/.ssh

id_rsa
id_rsa.pub


This is what the private key looks like.

In [7]:
docker exec ssh_client cat /root/.ssh/id_rsa | head -n 10 | tail -n 5

VmbWU1czmYmqmQ5zCXFkywcITNJtR9FEKH/JLtXHqkGfqjOSpkcfTKN0kWxuMOHLsHMSxy
71DkR/Va4vt6is0nOnIPKg61qt504BJ0YSXH2/P6nm/S1rJn6VSNhvGTHj3y3II24Zi1ms
VlaHDlOM6MCYzUDa9/ENS7AYLH+My9m6GjrNDJ/okmYXrelubAn1GEgqLCCoWbKjp8DnU4
paMmf0Z4RcutM/u00o9umhqGgBkRkhg0CwpnOZmKBU9SPo4+wLEB4O0bopv7er5N56bH9i
zWdL63MlHvH7po9DQDmJfJFtA7n6wu5KjluEemAlAAAFiKmgj02poI9NAAAAB3NzaC1yc2


**Note** Only a few lines can be printed here because Githab, which I use to save these pages, sends me notifications that I have credentials leaks.

And here is what the public key looks like.

In [8]:
docker exec ssh_client cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDGhSr9028BMhKfWXLDx0Q9kQW49xAKjH2xQshmY9LSGizvZ6wWFXxuHkGMiJQbRF0CMdcWnrm/N5RAQpWtft/XGRpfds8o2YyDIChBECeCT56ERPeBfHDLIh3Ii2S6eR4+DCQkCrvpmsOt2mPFxdP0KLPws7Fga7sWNvEg5uitfpexy7ZRSiT+gj22lNrQbOdWZtZTVzOZiaqZDnMJcWTLBwhM0m1H0UQof8ku1ceqQZ+qM5KmRx9Mo3SRbG4w4cuwcxLHLvUORH9Vri+3qKzSc6cg8qDrWq3nTgEnRhJcfb8/qeb9LWsmfpVI2G8ZMePfLcgjbhmLWaxWVocOU4zowJjNQNr38Q1LsBgsf4zL2boaOs0Mn+iSZhet6W5sCfUYSCosIKhZsqOnwOdTiloyZ/RnhFy60z+7TSj26aGoaAGRGSGDQLCmc5mYoFT1I+jj7AsQHg7Ruim/t6vk3npsf2LNZ0vrcyUe8fumj0NAOYl8kW0DufrC7kqOW4R6YCU= root@0f0391a4e1a5


## Copying public key

On the server, list all the public keys that have access in the `/root/.ssh/authorised_keys` file. In our case, there would only be one line corresponding to the generated pre-public key.

In [9]:
export public_key=$(docker exec ssh_client cat /root/.ssh/id_rsa.pub)
docker exec ssh_host mkdir /root/.ssh
docker exec ssh_host sh -c "echo \"$public_key\" >> /root/.ssh/authorized_keys"

## Getting acceess

If setup correcd you can get ssh from the client server to the host server using command `ssh <user_name>@<host_name>`. In our example try:

```bash
docker exec -it ssh_client ssh root@ssh_host
```

So here is a rather tricky case - we have access to the `ssh_host` CLI through the `ssh_client` CLI.

The following cell shows that it works: we create files in `ssh_host`using `ssh_client`.

In [10]:
docker exec ssh_client\
    ssh -o StrictHostKeyChecking=no root@ssh_host touch file{1..5}
docker exec ssh_host ls /root

file1
file2
file3
file4
file5


**Note** The `-o StrictHostKeyChecking=no` option is used here to prevent the client from asking if it needs to add host to the list of known hosts.